*Code by Claudia Corradino-INGV Catania.*\
*Reference: Corradino, C., Jouve, P., La Spina, A., & Del Negro, C. (2024). Monitoring Earth's atmosphere with Sentinel-5 TROPOMI and Artificial Intelligence: Quantifying volcanic SO2 emissions. Remote Sensing of Environment, 315, 114463.*

**Instructions**:
Run each cell except when you select volcano selection method (run only the cells related to the option you choose)
1) You will need to create a google and google earth engine account if not created yet. Click consense for use of Google services. Type your Google Earth Engine (GEE) project in the "ee_name" variable. YOU SHOULD BE LOGGED INTO COLAB WITH THE SAME GOOGLE ACCOUNT YOU SET FOR DRIVE AND GEE
2) Set the starting and ending date
3) Select one of the options to choose volcano/volcanoes to analyze among (1) from dropdown list,(2) by typing the name of the volcano/ list of volcanoesor (3) by selecting all volcanoes in a Region

The output files will be in the Drive folder: MyDrive/TROPOMI_RF:

- .png of the detected mask  (uncomment if you want to plot and save)
- .png  (uncomment if you want to plot and save), .geotiff and .nc of the detected so2
- Excel file containing so2 mass(kton) of the analyzed scenes, total of the scene and specific for the investigated volcano  


# *Libraries*

In [2]:
!pip install geemap
import geemap
from datetime import datetime
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
!pip install xarray
!pip install netCDF4
!pip install numpy
import netCDF4
import numpy as np
import xarray
from osgeo import gdal
import os
from skimage.measure import label, regionprops
from scipy.spatial.distance import pdist, squareform
!pip install geemap
import geemap
!pip install --force-reinstall gdal==3.6.4
from datetime import datetime
import glob
import matplotlib.image
from matplotlib import cm
import cv2
import rasterio as rs
from matplotlib import pyplot
from google.colab import drive
import ipywidgets as widgets
import logging
logging.getLogger("rasterio").setLevel(logging.ERROR)


  Using cached GDAL-3.6.4-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: gdal
    Found existing installation: GDAL 3.6.4
    Uninstalling GDAL-3.6.4:
      Successfully uninstalled GDAL-3.6.4


#*Input*

In [4]:
#@title GOOGLE AND GOOGLE EARTH ENGINE ACCOUNT { run: "auto", form-width: "50%" }

#insert your GOOGLE EARTH ENGINE project name
gee_project_name = 'victor-corradinoclaudia' #@param {type: "string"}

drive.mount('/content/drive')
path='/content/drive/MyDrive/TROPOMI_RF'#@param {type: "string"}

Mounted at /content/drive


In [93]:
#@title Insert dates of interest and volcano selection features { run: "auto", form-width: "50%" }

start_day = '2025-12-01' #@param {type: "date"}
end_day = '2025-12-03' #@param {type: "date"}

#insert volcano name/ names ['Shishaldin', 'Buldir']
volcano_selection_method=2#@param {type: "raw"}
volcano_name=['Etna', 'Stromboli'] #@param {type: "raw"}
volcano_region=' ' #@param {type: "string"}
radius= 600000 #@param {type: "raw"}


#*Functions Definition*



In [5]:
def featurecollection_to_dataframe(fc, batch_size=5000):
    """Converti FeatureCollection in pandas DataFrame.
    Se piccola, usa getInfo().
    Se grande, esporta su Drive e leggi CSV.
    """
    # Proviamo con getInfo() prima
    try:
        features = fc.getInfo()['features']
        data = [f['properties'] for f in features]
        df = pd.DataFrame(data)
        return df
    except Exception as e:
        # Esporta su Drive
        task = ee.batch.Export.table.toDrive(
            collection=fc,
            description='export_temp',
            fileFormat='CSV'
        )
        task.start()

        # Attendi che il task finisca
        while task.active():
            print("In corso...")
            time.sleep(5)

        return None

def CountBands(image):
  return image.set('count',image.bandNames().length())

def InputPreparation(image):
  #selecting needed GLCM bands form inital 18 ones
  bands_glcm = ['glcm_contrast','glcm_corr','glcm_var','glcm_savg']
  #bands used for Classification + select(add) bands from glcm
  bands = ee.List(['SO2_column_number_density','so2_15km','sd','entropy']).cat(ee.List(bands_glcm))

  # Kernel for Stdv and entropy
  bigKernel = ee.Kernel.square(
      radius = 4,
      units = 'pixels')

  #Kernel for Convolution
  l3 = [-0.4444,-0.4444, -0.1111, -0.4444,-0.4444]

  #Center of Kernel is zero
  c3 = [-0.1111, -0.1111,3.222, -0.1111,-0.1111]

  #Assemble a list of lists, the 9x9 kernel weights as a 2D matrix
  lists3 = [l3,l3,c3,l3,l3]
  kernel3 = ee.Kernel.fixed(5,5,lists3)

  #Kernel for focalMin/focalMax
  kernel = ee.Kernel.circle(radius = 2)

  ###First training image : '2022-07-27', '2022-07-28'
    #median to take average pixel value from remaning images of collection, multiply to convert into dobson unit
  SO2_at_1km = image.select('SO2_column_number_density').multiply(10000/4.4615)\

  SO2_at_15km = image.select('SO2_column_number_density_15km').multiply(10000/4.4615)\

  #Compute standard derivation (Texture)
  sd = SO2_at_1km.reduceNeighborhood(
   reducer= ee.Reducer.stdDev(),
   kernel= bigKernel
   ).reproject('EPSG:4326',None,3500)

  #Compute Entropy (Texture)
  intso2=SO2_at_1km.multiply(10000).int()
  entropy=intso2.entropy(bigKernel).reproject('EPSG:4326',None,3500)

  #Compute Graylevel Co-occurence matrix
  glcm = intso2.rename('glcm').glcmTexture(size= 1).reproject('EPSG:4326',None,3500).select(bands_glcm)#vor select!
  sd1 = sd.unmask(-100);
  entropy1 = entropy.unmask(-100);
  glcm1 = glcm.unmask(-100);

  input = SO2_at_1km.unmask(-100).rename('SO2_column_number_density').addBands(SO2_at_15km.rename('so2_15km')).addBands(sd1.rename('sd')).addBands(entropy1.rename('entropy')).addBands(glcm1)

  input = input.set('name',image.get('name')).set('Date', image.get('Date')).set('system:time_start', image.get('system:time_start'))

  date=image.get('system:time_start')
  start = ee.Date(date)
  end = start.advance(1, 'day');
  dataset =ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CLOUD').filterDate(start,end).filterBounds(study_zone).select('cloud_top_height').max().unmask(-100)
  return image.select('SO2_column_number_density').rename('in').addBands(input).addBands(dataset.rename('altitude')).addBands(SO2_at_1km.rename('input'))

def fromgeecollectiontonumpy(ee_object,out_dir,scale=None, crs=None, region=None, file_per_band=False
):
    """Exports an ImageCollection as GeoTIFFs.

    Args:
        ee_object (object): The ee.Image to download.
        out_dir (str): The output directory for the exported images.
        scale (float, optional): A default scale to use for any bands that do not specify one; ignored if crs and crs_transform is specified. Defaults to None.
        crs (str, optional): A default CRS string to use for any bands that do not explicitly specify one. Defaults to None.
        region (object, optional): A polygon specifying a region to download; ignored if crs and crs_transform is specified. Defaults to None.
        file_per_band (bool, optional): Whether to produce a different GeoTIFF per band. Defaults to False.
    """

    if not isinstance(ee_object, ee.ImageCollection):
        print("The ee_object must be an ee.ImageCollection.")
        return

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    try:

        count = int(ee_object.size().getInfo())
        print(f"Total number of images: {count}\n")
        print(f"Processing:")
        img_stack = []
        for i in range(0, count):
            image = ee.Image(ee_object.toList(count).get(i))
            image_array = geemap.ee_to_numpy(image.select('classification','SO2corrected'), region) # Client side
            img_stack.append(image_array)

        # Normalize array shapes and get min shape across all arrays (height, width)
        min_h = min(arr.shape[0] for arr in img_stack)
        min_w = min(arr.shape[1] for arr in img_stack)

        # Crop or pad each array to (min_h, min_w)
        normalized_stack = []
        for arr in img_stack:
            h, w = arr.shape[:2]
            # Crop larger arrays
            arr_cropped = arr[:min_h, :min_w, ...]
            # Pad smaller ones if needed (not strictly necessary here)
            if h < min_h or w < min_w:
                pad_h = min_h - h
                pad_w = min_w - w
                arr_cropped = np.pad(arr_cropped,((0, pad_h), (0, pad_w), (0, 0)),mode='constant',constant_values=np.nan,)
            normalized_stack.append(arr_cropped)
        return normalized_stack

    except Exception as e:
        print(e)

def apply_classifier(image3,ee_classifier):
  alt=image3.select('altitude')
  image2=image3.select(['SO2_column_number_density','so2_15km','sd','entropy','glcm_contrast','glcm_corr','glcm_var','glcm_savg']);
  image=image2.subtract(means).divide(std)
  so2_plume_15km=image3.select('so2_15km');
  so2_plume=image3.select('SO2_column_number_density')
  so2_median_density = so2_plume.unmask(-100);
  so2_classify_mask = image.classify(ee_classifier)
  so2_classify = so2_median_density.updateMask(so2_classify_mask)

  # Post-treatment : Apply a closing (dilatation = focalMax then erosion = focalMin)
  # Define a circle kernel of 1 pixel radius (adjust if needed, try 1.5)
  kernel = ee.Kernel.circle(
      radius = 1,
      units = 'pixels')
  connectivity=50;
  #Closing is better than opening to fullfil the plume and then apply PixelCount
  #2 iterations of dilatation and 3 of erosion seems to give the best results (adjust if needed)
  closed = so2_classify_mask.focalMax(
   kernel= kernel,
   iterations= 2).focalMin(
   kernel= kernel,
   iterations= 3).reproject('EPSG:4326',None,3500);
  so2_closed = so2_median_density.updateMask(closed)

  # Connectivity condition : to delete isolated pixels and keep only pixels part of the plume
  # ConnectedPixelCount returns a connection score of the SO2-positive pixels
  connexions = closed.selfMask().connectedPixelCount(ee.Number(connectivity).add(1)).reproject('EPSG:4326',None,3500);
  proxy = 0
  connexions = connexions.unmask(proxy)

  # Only keep the pixels that have a connectivity high enough
  # Score connectivity of 80 seems to be a good value, adjust if needed
  groupesPixels = connexions.gt(connectivity);
  positive_pixel = groupesPixels.reproject('EPSG:4326',None,3500);

  # Update the mask to the so2 map
  so2_plume = so2_median_density.updateMask(positive_pixel)
  # Update the mask to the so2 map
  so2_plume_15km = image.select('so2_15km').updateMask(positive_pixel)

  m2=positive_pixel.multiply(alt)
  m3=m2.updateMask(m2.gt(-100));
  mmean=ee.Image(ee.Number(m2.reduceRegion(reducer= ee.Reducer.max(),geometry= study_zone,scale= res,maxPixels= 1e13).values().get(0)));
  altcorreted=(positive_pixel.multiply(alt)).selfMask();
  m=m2.where(m2.eq(-100),mmean)
  flag=(so2_plume_15km.subtract(so2_plume)).multiply(m.subtract(10000)).divide(9000).add(so2_plume_15km);
  so2_corrected=so2_plume_15km.where(so2_plume.gte(0),flag.where(m.gt(10000),so2_plume_15km)).multiply(4.4615/10000).multiply(ee.Image.pixelArea()).multiply(0.000001*64*0.001)#
  st2=ee.Geometry.Polygon([[[158.6128787089055, 55.34598773699004],[158.6128787089055, 53.8505541387213],[164.4246462870305, 53.8505541387213],[164.4246462870305, 55.34598773699004]]])#, null, false);
  so2_original=so2_plume_15km#

  # Convert VCD from DU to mol/m2 (1DU=4.4615*10^-4mol/m2)
  kton=so2_corrected.reduceRegion(
          reducer= ee.Reducer.sum(),
          geometry= so2_corrected.geometry(),
          scale= res,
          maxPixels= 1e13
  ).values().get(0);

  output = image3.select('in').addBands(so2_plume).addBands(so2_plume_15km).addBands(positive_pixel.rename('classification')).addBands(m.selfMask().rename('masked_alt')).addBands(so2_original.rename('input')).addBands(image3.select('altitude')).addBands(so2_corrected.rename('SO2corrected')).set('name',image3.get('name')).set('kton',kton).set('Date', image3.get('Date')).set('system:time_start', image3.get('system:time_start'))

      #m.rename('masked_alt')).addBands(so2_corrected.rename('SO2corrected')).addBands(alt.rename('altitude'))
      #.set('kton',kton).set('Total_mass_corrected',total_mass111).set('Total_mass_1km',total_mass1).set('Total_mass_15km',total_mass15)//.addBands(ee.Image.constant(total_mass).rename('Total_mass'))

  return output
  #return classified.set('system:time_start', image.get('system:time_start'))

# Function to generate a list of the dates of the time series
def dayOffset(day):
  return startDate.advance(day, 'day')

def generateDateList(startDate, endDate):
  #Calculate the date range in milliseconds
  timeDiff = endDate.millis().subtract(startDate.millis());
  #Convert the difference in days
  daysDiff = timeDiff.divide(1000 * 60 * 60 * 24);
  dateList = ee.List.sequence(0, daysDiff.toInt());
  #Calculate the date from the startDate
  dateRange = dateList.map(dayOffset);
  return dateRange;

def One_image_per_dayfun(date):
    start = ee.Date(date)
    end = start.advance(1, 'day')

    filtered = raw.filterDate(start, end).filterBounds(study_zone)
    first_image = filtered.first()

    # Se non c'è immagine, ritorna un'immagine vuota (nulla)
    return ee.Algorithms.If(
        first_image,
        filtered \
            .select(['SO2_column_number_density', 'SO2_column_number_density_15km']) \
            .median() \
            .clip(study_zone) \
            .set('system:time_start', first_image.get('system:time_start')) \
            .set('name', ee.Date(first_image.get('system:time_start')).format('dd-MM-YYYY'))\
            .set('Date', ee.Date(first_image.get('system:time_start')).format('dd/MM/YYYY')),
        None
    )
def add_time_dim(xda):
     xda = xda.expand_dims(time = [datetime(2024,12,1)])
     return xda





# *GEE authentication*
Accessing Google Earth Engine

In [7]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project=gee_project_name)

#Load the trained classifier and normalization coefficients*
assetId = 'projects/victor-corradinoclaudia/assets/RF_Tropomi'
std = ee.Image('projects/victor-corradinoclaudia/assets/means_tropomi');
means = ee.Image('projects/victor-corradinoclaudia/assets/std_tropomi');
fc = ee.FeatureCollection('projects/victor-corradinoclaudia/assets/VolcanoList')
df = featurecollection_to_dataframe(fc)

#*Select one option to select volcanoes based on the variable "volcano_selection_method":*

1.   from dropdown list
2.   by typing the name of the volcano/ list of volcanoes
3.   by selecting all volcanoes in a Region



In [96]:
match volcano_selection_method:
    case 1:
        names = df['Volcano_Name'].dropna().tolist()
        # Crea il menu a tendina
        dropdown = widgets.Dropdown(
            options=sorted(names),
            description='Name:',
        )

        # Bottone
        button = widgets.Button(description="Submit")

        # Output
        output = widgets.Output()

        # Callback
        def on_click(b):
            with output:
                output.clear_output()
                print(f"Selected volcano: {dropdown.value}")
        button.on_click(on_click)
        display(dropdown, button, output)
    case 2:
        names = df['Volcano_Name'].dropna().tolist()
        non_valid = [p for p in volcano_name if p not in names]
        if non_valid:
            raise ValueError(f"Non valid names: {', '.join(non_valid)}")
    case 3:
        df_Alaska=df[df['Region']==volcano_region]
        volcano_name = df_Alaska['Volcano_Name'].dropna().tolist()


In [97]:
if volcano_selection_method==1:
  volcano_name=[dropdown.value]
print(volcano_name)

['Etna', 'Stromboli']


#*Main Code*


In [98]:
for i in range(len(volcano_name)):
  df2=df.loc[df['Volcano_Name']==volcano_name[i]]
  images_dir = os.path.join(path, volcano_name[i])
  os.makedirs(images_dir, exist_ok=True)
  print(f"-------------------------------")
  print(volcano_name[i])
  print(f"-------------------------------")
  latitude=df2.iloc[0]['Latitude']
  longitude=df2.iloc[0]['Longitude']
  volcano = ee.Geometry.Point([longitude,latitude])
  study_zone = volcano.buffer(radius)
  startDate = ee.Date(start_day);
  endDate = ee.Date(end_day)
  raw=ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_SO2").select('SO2_column_number_density','SO2_column_number_density_15km').filterBounds(study_zone).filterDate(startDate,endDate.advance(1, 'day'))
  dateList = generateDateList(startDate, endDate);
  One_image_per_day = ee.ImageCollection(dateList.map(One_image_per_dayfun)).map(CountBands)
  res=1113.2
  s5 = One_image_per_day \
      .filter(ee.Filter.eq('count', 2)) \
      .filter(ee.Filter.neq('Date', '23/03/2025')) \
      .map(InputPreparation)

  timestamps = One_image_per_day.aggregate_array('system:time_start').getInfo()
  date_strings = [datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps]

  # Apply the model
  ee_classifier = ee.Classifier.load(assetId)
  classified = s5.map(lambda image: apply_classifier(image, ee_classifier))
  ktonList = classified.aggregate_array('kton');

  classified.sort('system:time_start')
  #Lista di immagini
  image_list = classified.toList(classified.size())

  #Creazione file Excel
  workbook = openpyxl.Workbook()
  worksheet= workbook.active
  worksheet.append(['Data', 'kton','kton_volcano'])

  dates=[];
  values=[];

  #Estrazione valori in kton
  for ii in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(ii))
    name=image.get('name').getInfo()
    image2=image.addBands((image.select('classification').where(1,0)).paint(volcano.buffer(70000),1).rename('volcano'))
    geemap.ee_export_image(image2.select('SO2corrected','masked_alt','classification','volcano'), filename=f"{images_dir}/{name}.tif", scale=res, region=study_zone, file_per_band=False)
    raster = rs.open(f"{images_dir}/{name}.tif", "r+")
    array = raster.read()
    image_array=array[3]
    im = array[0]
    mask = array[2]
    kernel2 = np.ones((60,60), np.uint8)
    img_dilation = cv2.dilate(mask, kernel2, iterations=1)
    labeled_mask = label(np.squeeze(img_dilation))
    props = regionprops(labeled_mask)
    tot=np.squeeze(mask+image_array)
    kton2=0
    so2_corrected2=np.zeros(np.shape(im))

    for ii in range(len(props)+1):
        if np.max(tot*(labeled_mask==ii))==2:
          print('plume detected')
          mask_volcano=(labeled_mask==ii)*(mask)
          so2_corrected2=im*mask_volcano
          kton2=np.nansum(so2_corrected2)
          print(kton2)

          """plt.imshow((labeled_mask==ii)*(mask))
          plt.savefig(f"{images_dir}/{name}_mask.png")
          plt.show()"""

          """plt.imshow(so2_corrected2)
          plt.savefig(f"{images_dir}/{name}_so2.png")
          plt.show()"""

    kton_value = image.get('kton').getInfo()
    kton_value2 = kton2
    date_value = image.get('Date').getInfo()
    worksheet.append([name, kton_value, kton_value2])
    print(f"Data: {name}, kton_total: {kton_value}, kton_volcano: {kton_value2}")
    dates.append((datetime.strptime(date_value, "%d/%m/%Y")))
    values.append((kton_value))
    with rs.open(f"{images_dir}/{name}.tif", "r+") as update:
      update.write(so2_corrected2,4)
  #Salvataggio file
  file_name = f"{images_dir}/kton_dati.xlsx"
  workbook.save(file_name)

  print("Dati salvati in:", file_name)

-------------------------------
Etna
-------------------------------
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/TROPOMI_RF/Etna/01-12-2025.tif
plume detected
0.11244643354756549
Data: 01-12-2025, kton_total: 0.1303649925118221, kton_volcano: 0.11244643354756549
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/TROPOMI_RF/Etna/02-12-2025.tif
plume detected
0.05089566432972987
Data: 02-12-2025, kton_total: 0.30563207669517245, kton_volcano: 0.05089566432972987
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/TROPOMI_RF/Etna/03-12-2025.tif
Data: 03-12-2025, kton_total: 0, kton_volcano: 0
Dati salvati in: /content/drive/MyDrive/TROPOMI_RF/Etna/kton_dati.xlsx
-------------------------------
Stromboli
-------------------------------
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/TROPOMI_RF/Stromboli/01-12-2025.tif
plume detected
0.11244643354756552
Data: 01-12-2025, kton_tot

#Map


In [99]:
  filtered=classified.filterMetadata('kton','greater_than',0)
  # Compute statistics (sum in this case)
  stat=filtered.sum()

  #geom = classified.first().geometry()
  #bounds = geom.bounds().getInfo()['coordinates'][0]
  box=ee.Geometry.Polygon([[[14.886997380371172, 5.824280032173915],[14.886997380371172, -6.93674077038431],[32.64090363037117, -6.93674077038431],[32.64090363037117, 5.824280032173915]]]);
  vis_params2 = {
    'min': 0,
    'max': 0.005,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1,
  };

  vis_params1 = {
    'min': 0,
    'max': (filtered.size().getInfo()),
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1,
  };

  geemap.update_package()
  Map = geemap.Map(center=(latitude,longitude), zoom=6)
  Map.add_basemap("HYBRID")
  Map.add_layer(stat.select('classification').selfMask(), vis_params1, ' Occurrencies')
  Map.add_layer(stat.select('SO2corrected'), vis_params2, ' Cumulative')
  Map.addLayer(volcano, {'color': 'black'}, 'Pin')

  Map

Unzipping geemap-master.zip ...
Data downloaded to: /root/Downloads/geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


Map(center=[np.float64(38.78900146484375), np.float64(15.213000297546387)], controls=(WidgetControl(options=['…